In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import multiprocessing

Using TensorFlow backend.


In [2]:
import importlib
importlib.reload(agent)
importlib.reload(environment)
importlib.reload(doubledqn)
importlib.reload(tools)
importlib.reload(memory)

<module 'memory' from '/Users/seb/Documents/GitHub/MS_Transports_RL/Scripts/Tests_Sumo/cross_RL/memory.py'>

In [3]:
# MAIN
##################################

num_actions = 2
state_shape = (1,11) # State var in rows
memory_size = 20000
gamma = 0.9
target_update_frequency = 30000
num_init_samples_mem = 1000
batch_size = 50
max_episode_length = 10000
optimizer = 'adam'
loss = "mse"
eps = 0.1
env_name = "Simple_Cross"
experiment_id = "Test_linear_model"
monitoring = True # Store variables for TensorBoard monitoring and model_checkpoints

# Define logs directory if monitoring enabled
if monitoring:
    output_dir = tools.get_output_folder("./Logs",experiment_id)
    summary_writer = tf.summary.FileWriter(logdir=output_dir)
else:
    output_dir = None
    summary_writer = None

# Initialize Q-networks (value and target)
q_network = agent.get_model('simple',(state_shape[1],),num_actions)
target_q_network = agent.get_model('simple',(state_shape[1],),num_actions)

# Initialize environment
sumo_env =  environment.Env(    "cross.net.xml",
                                "cross.rou.xml",
                                state_shape,
                                num_actions,
                                use_gui=False
                           )

# Initialize replay memory
mem = memory.ReplayMemory(    memory_size,
                                 state_shape,
                                 num_actions
                             )

# Initialize Double DQN algorithm
ddqn = doubledqn.DoubleDQN(     q_network,
                                target_q_network,
                                mem,
                                gamma,
                                target_update_frequency,
                                num_init_samples_mem,
                                batch_size,
                                optimizer,
                                loss,
                                max_episode_length,
                                sumo_env,
                                output_dir,
                                experiment_id,
                                summary_writer
                            )

# Fill Replay Memory
ddqn.fill_replay(sumo_env)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/seb/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-cf7dee10b5c0>", line 28, in <module>
    q_network = agent.get_model('simple',(state_shape[1],),num_actions)
TypeError: get_model() takes 1 positional argument but 3 were given

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/seb/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/seb/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_

TypeError: get_model() takes 1 positional argument but 3 were given

In [ ]:
# Train
_ = ddqn.train(  sumo_env, 2, "epsGreedy", eps=eps)

In [ ]:
def get_chunks(iterable, chunks=8):
    """Split parameter grid into chunks"""
    lst = list(iterable)
    grid_chunks = [["Worker #" + str(i + 1), lst[i::chunks]] for i in range(chunks)]
    return grid_chunks

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(itertools.islice(iterable, n))

def worker(chunked_param_list):
    best_run_time = 100000
    for params in chunked_param_list:
        
        # Initialize Q-networks (value and target)
        q_network = agent.get_model('simple',(state_shape[1],),num_actions)
        target_q_network = agent.get_model('simple',(state_shape[1],),num_actions)

        # Initialize environment
        sumo_env =  environment.Env(    "cross.net.xml",
                                "cross.rou.xml",
                                state_shape,
                                num_actions,
                                use_gui=False
                           )

        # Initialize replay memory
        mem = memory.ReplayMemory(    memory_size,
                                 state_shape,
                                 num_actions
                             )
        
        # Initialize Double DQN algorithm
        ddqn = doubledqn.DoubleDQN(     q_network = q_network,
                                target_q_network = target_q_network,
                                memory = mem,
                                gamma = params[2],
                                target_update_freq = params[1],
                                num_burn_in = num_init_samples_mem,
                                batch_size = params[0],
                                optimizer = optimizer,
                                loss_func = loss,
                                max_ep_length = max_episode_length,
                                env_name = sumo_env,
                                output_dir = output_dir,
                                experiment_id = experiment_id,
                                summary_writer = summary_writer)
        
        # Fill Replay Memory
        ddqn.fill_replay(sumo_env)
        
        # Train
        _ = ddqn.train(sumo_env, 2, "epsGreedy", eps = params[3])
        
        run_time = ddqn.evaluate_cv(sumo_env, "greedy")
        if run_time < best_run_time:
            best_run_time = run_time
            best_params = params
            
    return best_run_time

def gridsearch(param_grid = param_grid):
    jobs = []
    chunked_param_list = get_chunks(param_grid, chunks = multiprocessing.cpu_count())
    pool = multiprocessing.Pool()
    results = pool.map(worker, chunked_param_list)
    pool.close()
    pool.join()
    print(results)
    # Now combine the results
    sorted_results = reversed(sorted(results, key=lambda x: x[0]))
    return(sorted_results)  # Winner

In [ ]:
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
target_update_frequency = [10, 50, 100]
gamma = [0.6, 0.7, 0.8, 0.9]
eps = [0.1, 0.2]

param_grid = itertools.product(batch_size, target_update_frequency, gamma, eps)

In [ ]:
test = get_chunks(param_grid)

In [ ]:
test[0][1]

In [ ]:
check = (test[0], workers[0])

In [ ]:
check[1][]

In [ ]:
pool = multiprocessing.Pool()

In [ ]:
pool.map()

In [ ]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(itertools.islice(iterable, n))

In [ ]:
take(20, param_grid)

In [ ]:
gridsearch()

In [ ]:
tools.generate_routefile()
sumo_env =  environment.Env(    "cross.net.xml",
                                "cross.rou.xml",
                                state_shape,
                                num_actions,
                                use_gui=False
                           )
ddqn.train(  sumo_env, 1, "epsGreedy", eps=eps)
tools.compute_mean_duration(output_dir)

In [ ]:
import pandas as pd
pd.DataFrame(data)

In [ ]:
pd.DataFrame(data).state[22]

In [ ]:
numberList = [1, 2, 3]
strList = ['one', 'two', 'three']

# No iterables are passed
result = zip()

# Converting itertor to list
resultList = list(result)
print(resultList)

In [ ]:
class test:
    def __init__(self):
        self.number = 5
        
    def reset(self):
        self.__init__()
        
    def high(self):
        self.number = 10

In [ ]:
check = test()

In [ ]:
check.number

In [ ]:
check.high()
check.number

In [ ]:
check.reset()

In [ ]:
check.number